In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# =========================
#  STEP 1 — LOAD DATASET
# =========================
df = pd.read_csv('historical_data.csv')

df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

print("✅ Dataset loaded successfully!")
print(df.head())

# =========================
#  STEP 2 — FEATURE ENGINEERING
# =========================
df['High-Low'] = df['High'] - df['Low']
df['Open-Close'] = df['Open'] - df['Close']
df['7day MA'] = df['Close'].rolling(7).mean()
df['14day MA'] = df['Close'].rolling(14).mean()
df.dropna(inplace=True)

features = df[['Close', 'Volume', 'High-Low', 'Open-Close', '7day MA', '14day MA']].values

# =========================
#  STEP 3 — SCALE FEATURES
# =========================
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(features)

# =========================
#  STEP 4 — CREATE SEQUENCES
# =========================
def create_sequences(data, seq_len):
    X, y = [], []
    for i in range(seq_len, len(data)):
        X.append(data[i-seq_len:i])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

seq_len = 60  
X, y = create_sequences(scaled_data, seq_len)

print(f"📊 Total sequences created: {len(X)}")

# Train/Test split
train = int(len(X) * 0.8)
X_train, X_test = X[:train], X[train:]
y_train, y_test = y[:train], y[train:]

print(f"📌 Train Size: {len(X_train)} | Test Size: {len(X_test)}")

# =========================
#  STEP 5 — BUILD LSTM MODEL
# =========================
model = Sequential([
    Bidirectional(LSTM(128, return_sequences=True), input_shape=(seq_len, X.shape[2])),
    LayerNormalization(),
    Dropout(0.3),

    Bidirectional(LSTM(128, return_sequences=True)),
    LayerNormalization(),
    Dropout(0.3),

    Bidirectional(LSTM(64)),
    LayerNormalization(),
    Dropout(0.2),

    Dense(32, activation='relu'),
    Dense(1)
])

optimizer = Adam(learning_rate=0.0005, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='huber')

# Callbacks
stopper = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
saver = ModelCheckpoint("best_model.keras", monitor='val_loss', save_best_only=True)

# =========================
#  STEP 6 — TRAIN MODEL
# =========================
print("\n🚀 Training Started...\n")
model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[stopper, saver]
)

# =========================
#  STEP 7 — LOAD BEST MODEL
# =========================
from tensorflow.keras.models import load_model
model = load_model("best_model.keras")

# =========================
#  STEP 8 — PREDICT & GENERATE TREND
# =========================
pred = model.predict(X_test)

dummy = np.zeros((len(pred), scaled_data.shape[1]))
dummy[:, 0] = pred[:, 0]

pred_prices = scaler.inverse_transform(dummy)[:, 0]

trend = np.where(pred_prices[1:] > pred_prices[:-1], "Up", "Down")

print("\n📈 Trend Output Sample:")
for i in range(10):
    print(f"Day {i+1}: {trend[i]}")

# Save final trained model
model.save("trend_model.keras")
print("\n✅ Model saved as trend_model.keras")

# =========================
#  STEP 9 — FUTURE TREND FUNCTION
# =========================
def predict_next_trend(model, df, data, seq_len=60):
    X_input = data[-seq_len:].reshape(1, seq_len, data.shape[1])
    pred_price = model.predict(X_input)

    dummy = np.zeros((1, data.shape[1]))
    dummy[:, 0] = pred_price
    next_price = scaler.inverse_transform(dummy)[0, 0]

    today = df['Close'].iloc[-1]
    return "Up" if next_price > today else "Down"

next_trend = predict_next_trend(model, df, scaled_data)
print("\n🔮 Tomorrow Trend:", next_trend)


✅ Dataset loaded successfully!
              Open    High     Low   Close   Volume
Date                                               
2025-01-01  100.00  101.58   99.56  101.55  4718635
2025-01-02  101.55  101.64  100.16  100.84  2670478
2025-01-03  100.84  102.43  100.79  101.51  4840580
2025-01-04  101.51  102.68  101.18  102.36  2830051
2025-01-05  102.36  102.40  100.81  101.80  1654576
📊 Total sequences created: 127
📌 Train Size: 101 | Test Size: 26


c:\Users\kush2\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



🚀 Training Started...

Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 422ms/step - loss: 0.3328 - val_loss: 0.0536
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step - loss: 0.0970 - val_loss: 0.0261
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - loss: 0.0771 - val_loss: 0.0021
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 128ms/step - loss: 0.0558 - val_loss: 0.0312
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step - loss: 0.0246 - val_loss: 0.0405
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step - loss: 0.0210 - val_loss: 0.0706
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - loss: 0.0140 - val_loss: 0.0723
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - loss: 0.0120 - val_loss: 0.0702
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 128ms/step - loss: 0.0138 - val_loss: 0.0713
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step - loss: 0.0076 - val_loss: 0.0774
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 758ms/step

📈 Trend Output Sample:
Day 1: Down
Day 2: Up
Day 3: Up
Day 4: Up
Day 5: Up
Day 6: Down
Day 7: 